In [1]:
require 'nngraph';
require 'torch';

function getCtrainExample(set, labels)

    -- randomly select two images from different classes
    torch.manualSeed(3489208)
    local y = torch.randperm((#set)[1])

    local im_feat = {}
    im_feat[1] = set[y[1]]
    im_feat[2] = set[y[2]]

    -- randomly select one of those to be given to model B
    local label = torch.bernoulli() + 1
    im_feat[3] = im_feat[label]:clone()

    -- randomly select one of the images to ask about its class
    local ques = torch.Tensor(42):fill(0)
    ques[labels[y[torch.bernoulli() + 1]]] = 1

    -- target is probability of label = 2
    local target = label - 1
    local input = torch.cat({im_feat[1], im_feat[2], im_feat[3], ques})
    
    return {input, target}
end


function nextBatch(trainset, train_labels, batchSize)
    local inputs = torch.Tensor(batchSize, 12330);
    local targets = torch.Tensor(batchSize);
    local i = 0;
    for i = 1, batchSize do
        example = getCtrainExample(trainset, train_labels);
        inputs[i] = example[1];
        targets[i] = example[2];
    end
    inputs:double();
    targets:double();
    return inputs, targets
end

In [2]:
-- read preprocessed feature vectors and labels
feat_vecs = torch.load('feat_vecs.t7')
labels = torch.load('labels.t7')

-- generate trainset and testset
train_perc = 0.80 -- percentage of images in the train set
trainset_size = torch.round((#feat_vecs)[1] * train_perc)
trainset = feat_vecs[{{1, trainset_size}}]
train_labels = labels[{{1, trainset_size}}]
testset = feat_vecs[{{trainset_size+1, (#feat_vecs)[1]}}]
test_labels = labels[{{trainset_size+1, (#feat_vecs)[1]}}]

In [36]:
C_model_old = torch.load('C_model__1500.t7')

C_model = torch.load('C_model.t7')
C_model.modules[13].modules[1].weight = C_model_old.modules[10].modules[1].weight:clone()
C_model.modules[13].modules[3].weight = C_model_old.modules[10].modules[3].weight:clone()
C_model.modules[13].modules[1].bias = C_model_old.modules[10].modules[1].bias:clone()
C_model.modules[13].modules[3].bias = C_model_old.modules[10].modules[3].bias:clone()

In [37]:
C_model:evaluate()
C_model_old:evaluate()

C_model:double()
C_model_old:double()

In [38]:
-- confirm that the weights are the same


In [39]:
-- All 4096x42 modules
a1 = C_model_old.modules[2].modules[3]
a2 = C_model_old.modules[6].modules[3]
a3 = C_model.modules[2].modules[3]
a4 = C_model.modules[7].modules[3]
print(torch.max(a1.weight - a2.weight))
print(torch.max(a2.weight - a3.weight))
print(torch.max(a3.weight - a4.weight))
print(torch.max(a4.weight - a1.weight))
print(torch.max(a1.bias - a2.bias))
print(torch.max(a2.bias - a3.bias))
print(torch.max(a3.bias - a4.bias))
print(torch.max(a4.bias - a1.bias))

-- All 3x2 modules
b1 = C_model_old.modules[10].modules[1]
b2 = C_model.modules[13].modules[1]
print(torch.max(b1.weight - b2.weight))
print(torch.max(b1.bias - b2.bias))

-- All 2x1 modules
c1 = C_model_old.modules[10].modules[3]
c2 = C_model.modules[13].modules[3]
print(torch.max(c1.weight - c2.weight))
print(torch.max(c1.bias - c2.bias))

0	


0	


0	


0	
0	
0	
0	
0	
0	
0	
0	
0	


In [40]:
batch_size = 1
inputs, targets = nextBatch(trainset, train_labels, batch_size);
input = inputs[1]

image_feat1 = input:narrow(1, 1, 4096)
image_feat2 = input:narrow(1, 4097, 4096)
image_feat3 = input:narrow(1, 8193, 4096)
question = input:narrow(1, 12289, 42)
confidence = torch.Tensor{0.8}

print(C_model:forward({image_feat1, image_feat2, question, confidence}))
print(C_model_old:forward({image_feat1, image_feat2, question, confidence}))

1e-16 *
  2.4476
[torch.DoubleTensor of size 1x1]

1e-16 *
 2.4476
[torch.DoubleTensor of size 1]

